In [1]:
import chardet

with open('./전기차데이타/전기차_통합.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))  # 일부만 분석
    print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [2]:
import pandas as pd

df = pd.read_csv('./전기차데이타/전기차_통합.csv', encoding='utf-8')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11772 entries, 0 to 11771
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연월      11772 non-null  float64
 1   지역      11772 non-null  object 
 2   연료유형    11772 non-null  object 
 3   용도      11772 non-null  object 
 4   등록대수    11691 non-null  float64
dtypes: float64(2), object(3)
memory usage: 460.0+ KB


In [3]:
df.head()

,연월,지역,연료유형,용도,등록대수
0,2017.01,강원,경유,사업용,12362.0
1,2017.01,광주,전기,비사업용,240.0
2,2017.01,인천,전기,사업용,6.0
3,2017.01,인천,전기,비사업용,210.0
4,2017.01,대구,전기,사업용,53.0


In [5]:
df.rename(columns={
    '연월': 'datetime',
    '지역': 'region',
    '연료유형': 'fuel_type',
    '용도': 'use_type',
    '등록대수': 'reg_count'
}, inplace=True)


# 결과 확인
print(df.columns)

Index(['datetime', 'region', 'fuel_type', 'use_type', 'reg_count'], dtype='object')


In [6]:
df.head()

,datetime,region,fuel_type,use_type,reg_count
0,2017.01,강원,경유,사업용,12362.0
1,2017.01,광주,전기,비사업용,240.0
2,2017.01,인천,전기,사업용,6.0
3,2017.01,인천,전기,비사업용,210.0
4,2017.01,대구,전기,사업용,53.0


In [7]:
import json
from sqlalchemy import create_engine 
from sqlalchemy.exc import SQLAlchemyError
import cryptography

with open('db-config.json', 'r') as f:
    config = json.load(f)

user = config['user']
password = config['password']
host = config['host']
port = config['port']
database = config['database']

df['datetime'] = pd.to_datetime(df['datetime'])  # datetime 형식으로 변환




In [8]:
# SQLAlchemy 엔진 생성
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}")
conn = engine.connect()
# 데이터 삽입 (예외처리 포함)
try:
    df.to_sql(name='vehicle_reg_stats', con=conn, if_exists='append', index=False)
    print("[Success] 데이터가 성공적으로 삽입되었습니다.")
except SQLAlchemyError as e:
    print("[Failure] 데이터 삽입 중 오류 발생:", str(e))


[Success] 데이터가 성공적으로 삽입되었습니다.
